api call

In [26]:
from openai import OpenAI
from tqdm import tqdm
import httpx
import pandas as pd
import openai
import backoff

api_key = "sk-5tlQk10jHxjUJ47iF387Ca4a18F04eDaAd5f2b6c90341e44"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

@backoff.on_exception(backoff.expo, (openai.APIStatusError, openai.InternalServerError), max_tries=5)
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    return response.choices[0].message.content.strip('\n')


instruction_noindex_old = '''You will be provided with some log messages. You should check whether the giving log messages share the same template. If so, abstract variables with `{placeholders}` and return the template without additional explatation'''

instruction_noindex = '''You will be provided with some log messages. You should check if the giving log messages share the same template. If so, abstract variables with `{{placeholders}}` to extract the corresponding template.
Print the input log's template delimited by backticks.
'''

instruction_100 = '''do the following log contents share the same template? Return the template or templates without additional explatation'''

instruction_1_13 = '''You will be provided with some log messages. You should check whether the giving log messages share the same template. If so, abstract variables with ‘{{placeholders}}’ and return the template without additional explatation'''

instruction_1_13_2 = '''Given some log templates, you should choice the most corret one. If you think none of them is correct, you can return the correct tempalte you think'''

prompt = '''Block broadcast_9 stored as values in memory (estimated size 8.8 KB, free 14.0 KB)'''

instruction_for_one_log = '''You will be provided with a log message delimited by backticks. You must abstract variables with `{placeholders}` to extract the corresponding template.
Print the input log's template delimited by backticks.'''

input = '''Log message: `Block broadcast_9 stored as values in memory (estimated size 8.8 KB, free 14.0 KB)`'''
# ready=true,policy=3,wakefulness=1,wksummary=0x23,uasummary=0x1,bootcompleted=true,boostinprogress=false,waitmodeenable=false,mode=false,manual=38,auto=-1,adj=0.0userId=0
prompt2 = '''Finished task 3.0 in stage 0.0 (TID 3). 2703 bytes result sent to driver
Finished task 1.0 in stage 0.0 (TID 1). 2703 bytes result sent to driver
Finished task 2.0 in stage 0.0 (TID 2). 2703 bytes result sent to driver
Finished task 4.0 in stage 0.0 (TID 4). 2703 bytes result sent to driver
Finished task 0.0 in stage 0.0 (TID 0). 2703 bytes result sent to driver
Finished task 4.0 in stage 1.0 (TID 79). 2685 bytes result sent to driver
Finished task 1.0 in stage 1.0 (TID 56). 2428 bytes result sent to driver
Finished task 0.0 in stage 1.0 (TID 42). 2474 bytes result sent to driver
Finished task 3.0 in stage 1.0 (TID 78). 2651 bytes result sent to driver
Finished task 2.0 in stage 1.0 (TID 69). 2523 bytes result sent to driver
Finished task 4.0 in stage 2.0 (TID 119). 2087 bytes result sent to driver
Finished task 2.0 in stage 2.0 (TID 112). 2087 bytes result sent to driver
Finished task 3.0 in stage 2.0 (TID 118). 2087 bytes result sent to driver
Finished task 1.0 in stage 2.0 (TID 101). 2087 bytes result sent to driver
Finished task 0.0 in stage 2.0 (TID 87). 2087 bytes result sent to driver
Finished task 4.0 in stage 3.0 (TID 159). 2667 bytes result sent to driver
Finished task 3.0 in stage 3.0 (TID 158). 2667 bytes result sent to driver
Finished task 1.0 in stage 3.0 (TID 136). 2667 bytes result sent to driver
Finished task 2.0 in stage 3.0 (TID 150). 2667 bytes result sent to driver
Finished task 0.0 in stage 3.0 (TID 122). 2667 bytes result sent to driver
Finished task 0.0 in stage 4.0 (TID 171). 2099 bytes result sent to driver
Finished task 1.0 in stage 4.0 (TID 185). 2099 bytes result sent to driver
Finished task 2.0 in stage 4.0 (TID 195). 2138 bytes result sent to driver
Finished task 3.0 in stage 4.0 (TID 198). 2099 bytes result sent to driver
Finished task 4.0 in stage 4.0 (TID 199). 2138 bytes result sent to driver
Finished task 0.0 in stage 5.0 (TID 206). 2198 bytes result sent to driver
Finished task 3.0 in stage 5.0 (TID 238). 2198 bytes result sent to driver
Finished task 2.0 in stage 5.0 (TID 231). 2198 bytes result sent to driver
Finished task 4.0 in stage 5.0 (TID 239). 2198 bytes result sent to driver
Finished task 1.0 in stage 5.0 (TID 220). 2272 bytes result sent to driver
Finished task 0.0 in stage 6.0 (TID 247). 2087 bytes result sent to driver
Finished task 3.0 in stage 6.0 (TID 278). 2087 bytes result sent to driver
Finished task 2.0 in stage 6.0 (TID 273). 2087 bytes result sent to driver
Finished task 4.0 in stage 6.0 (TID 279). 2087 bytes result sent to driver
Finished task 1.0 in stage 6.0 (TID 261). 2087 bytes result sent to driver
Finished task 0.0 in stage 7.0 (TID 285). 2667 bytes result sent to driver
Finished task 3.0 in stage 7.0 (TID 318). 2667 bytes result sent to driver
Finished task 4.0 in stage 7.0 (TID 319). 2667 bytes result sent to driver
Finished task 1.0 in stage 7.0 (TID 299). 2667 bytes result sent to driver
Finished task 2.0 in stage 7.0 (TID 312). 2667 bytes result sent to driver
Finished task 0.0 in stage 8.0 (TID 321). 2141 bytes result sent to driver
Finished task 2.0 in stage 8.0 (TID 349). 2141 bytes result sent to driver
Finished task 4.0 in stage 8.0 (TID 359). 2141 bytes result sent to driver
Finished task 3.0 in stage 8.0 (TID 358). 2141 bytes result sent to driver
Finished task 1.0 in stage 8.0 (TID 335). 2111 bytes result sent to driver
Finished task 0.0 in stage 9.0 (TID 360). 2239 bytes result sent to driver
Finished task 2.0 in stage 9.0 (TID 388). 2239 bytes result sent to driver
Finished task 3.0 in stage 9.0 (TID 398). 2239 bytes result sent to driver
Finished task 1.0 in stage 9.0 (TID 374). 2298 bytes result sent to driver
Finished task 4.0 in stage 9.0 (TID 399). 2239 bytes result sent to driver
Finished task 3.0 in stage 10.0 (TID 438). 2087 bytes result sent to driver
Finished task 0.0 in stage 10.0 (TID 402). 2087 bytes result sent to driver
Finished task 4.0 in stage 10.0 (TID 439). 2087 bytes result sent to driver
Finished task 2.0 in stage 10.0 (TID 430). 2087 bytes result sent to driver
Finished task 1.0 in stage 10.0 (TID 416). 2087 bytes result sent to driver
Finished task 4.0 in stage 11.0 (TID 479). 2667 bytes result sent to driver
Finished task 0.0 in stage 11.0 (TID 450). 2667 bytes result sent to driver
Finished task 3.0 in stage 11.0 (TID 478). 2667 bytes result sent to driver
Finished task 2.0 in stage 11.0 (TID 475). 2667 bytes result sent to driver
Finished task 1.0 in stage 11.0 (TID 464). 2667 bytes result sent to driver
Finished task 0.0 in stage 12.0 (TID 480). 2155 bytes result sent to driver
Finished task 3.0 in stage 12.0 (TID 518). 2155 bytes result sent to driver
Finished task 4.0 in stage 12.0 (TID 519). 2155 bytes result sent to driver
Finished task 1.0 in stage 12.0 (TID 494). 2105 bytes result sent to driver
Finished task 2.0 in stage 12.0 (TID 508). 2155 bytes result sent to driver
Finished task 0.0 in stage 13.0 (TID 528). 2256 bytes result sent to driver
Finished task 4.0 in stage 13.0 (TID 559). 2241 bytes result sent to driver
Finished task 2.0 in stage 13.0 (TID 553). 2241 bytes result sent to driver
Finished task 1.0 in stage 13.0 (TID 542). 2241 bytes result sent to driver
Finished task 3.0 in stage 13.0 (TID 558). 2257 bytes result sent to driver
Finished task 3.0 in stage 14.0 (TID 598). 2087 bytes result sent to driver
Finished task 2.0 in stage 14.0 (TID 590). 2087 bytes result sent to driver
Finished task 0.0 in stage 14.0 (TID 562). 2087 bytes result sent to driver
Finished task 4.0 in stage 14.0 (TID 599). 2087 bytes result sent to driver
Finished task 1.0 in stage 14.0 (TID 576). 2087 bytes result sent to driver
Finished task 1.0 in stage 15.0 (TID 626). 2667 bytes result sent to driver
Finished task 3.0 in stage 15.0 (TID 638). 2667 bytes result sent to driver
Finished task 4.0 in stage 15.0 (TID 639). 2667 bytes result sent to driver
Finished task 2.0 in stage 15.0 (TID 636). 2667 bytes result sent to driver
Finished task 0.0 in stage 15.0 (TID 612). 2667 bytes result sent to driver'''

messages = [{"role": "system", "content": instruction_noindex}]
messages.append({"role": "user", "content": prompt2})
print(get_responce(messages))

`Finished task {{task_id}} in stage {{stage_id}} (TID {{task_tid}}). {{bytes}} bytes result sent to driver`


In [9]:
from collections import Counter


def most_common(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]


# 测试
lst = [1, 2, 7, 4, 3, 3]
print(most_common(lst))

3


Evaluate_gpt

In [17]:
from evaluator import evaluate
import pandas as pd
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
m,n,p,q = [],[],[],[]
for dataset in datasets:
    file = f'outputs/k_means/initial/{dataset}.csv'
    # df = pd.read_csv(f'outputs/k_means/initial/{dataset}.csv')
    # df2 =
    a,b,c,d = evaluate(file, dataset)
    m.append(a)
    n.append(b)
    p.append(c)
    q.append(d)

print(sum(m)/len(m))
print(sum(n)/len(n))

Precision: 0.9998, Recall: 0.9993, F1_measure: 0.9995, Group Accuracy: 0.9495, Message-Level Accuracy: 0.8160, Edit Distance: 2.6080
Precision: 1.0000, Recall: 1.0000, F1_measure: 1.0000, Group Accuracy: 1.0000, Message-Level Accuracy: 0.2695, Edit Distance: 12.6970
Precision: 0.9999, Recall: 0.8151, F1_measure: 0.8981, Group Accuracy: 0.4375, Message-Level Accuracy: 0.6270, Edit Distance: 5.0305
Precision: 0.7545, Recall: 0.9993, F1_measure: 0.8598, Group Accuracy: 0.7315, Message-Level Accuracy: 0.6270, Edit Distance: 7.3525
Precision: 1.0000, Recall: 0.8777, F1_measure: 0.9349, Group Accuracy: 0.5345, Message-Level Accuracy: 0.3635, Edit Distance: 8.5830
Precision: 1.0000, Recall: 0.9383, F1_measure: 0.9682, Group Accuracy: 0.4340, Message-Level Accuracy: 0.3130, Edit Distance: 4.1120
Precision: 1.0000, Recall: 0.6128, F1_measure: 0.7599, Group Accuracy: 0.5265, Message-Level Accuracy: 0.0255, Edit Distance: 23.2700
Precision: 0.5527, Recall: 0.9998, F1_measure: 0.7119, Group Accura

验证按' '分词有多少词，以及含数字的词占多少

In [1]:
import re
import pandas as pd
datasets = ['BGL' ,'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac','Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
a = 0
b = 0
list = []
list2 = []
for dataset in datasets:
    df = pd.read_csv(f"dataset\{dataset}\{dataset}_2k.log_templates_corrected.csv")
    list = df['EventTemplate'].tolist()
    list2.append(len(list))    
    total_words = 0
    words_with_numbers = 0

    for s in list:
        words = s.split()
        total_words += len(words)
        words_with_numbers += len([word for word in words if re.search(r'\d', word)])
    a+=total_words
    b+=words_with_numbers

print(f"{b} / {a}")

109 / 9841


In [2]:
print(list2)

[120, 14, 116, 75, 43, 26, 8, 46, 50, 341, 114, 158, 50, 6, 149, 36]


In [2]:
import json
# load demonstrations
with open('demonstrations.json', 'r') as f:
    demonstrations = json.load(f)
messages = []
print(demonstrations['Spark'][0])

{'role': 'user', 'content': 'Log: Block broadcast_0 stored as values in memory (estimated size 384.0 B, free 317.5 KB)'}


聚类结果的自定义处理

In [22]:
import re

def replace_words(str):
    words = str.split(' ')
    print(words)
    for i in range(len(words)):
        if words[i].count('<*>') >= 2:
            if words[i].startswith('/'):
                temp = words[i][1:]
            # 保留前后的符号
            else:
                prefix = '' if not re.match(
                    r'^[\[\]\.\:\,\/\']', words[i]) else words[i][0]
                suffix = '' if not re.match(
                    r'.*[\[\]\.\:\,\/\']$', words[i]) else words[i][-1]
                temp = prefix + '<*>' + suffix
            str = str.replace(words[i], temp)
    return str

    # zookeeper
    # output_str = input_str.replace(' /<*>', ' <*>')

# 测试


inputs = '''Saved output of task ,attempt_<*>_m_<*>_r_<*>. to hdfs://<*>/pjhe/test/<*>/_temporary/0/task_<*>_m_<*>_r_<*>'''

lines = inputs.split('\n')
for line in lines:
    print(replace_words(line))

['Saved', 'output', 'of', 'task', ',attempt_<*>_m_<*>_r_<*>.', 'to', 'hdfs://<*>/pjhe/test/<*>/_temporary/0/task_<*>_m_<*>_r_<*>']
Saved output of task ,<*>. to <*>


In [33]:
from collections import Counter
list = ['1','1','1','3','3','3','4','4']
words = [word for word in list if re.search(r'\d', word)]
num = len(words)

freq = Counter(list)
print (len(freq))
print(freq.most_common(3))

3
[('1', 3), ('3', 3), ('4', 2)]


In [21]:
from openai import OpenAI
import httpx
client = OpenAI(
    base_url="https://4.0.996444.icu/v1", 
    api_key="sk-1XhqpmIwxu9nCuCD51FcA00948F54eDf9cA4765998Ce8f0d",
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)
message_list = [
     {"role": "user", "content": "Hello"}, 
    ]

response = client.chat.completions.create(
    model="gpt-4",
    messages=message_list,
    temperature=0,
    max_tokens=1024
)

print(response)
response = response.choices[0].message.content

ChatCompletion(id='chatcmpl-8gsegeGGp7R5X0cjWY4oLnFg1Rh66', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1705230562, model='gpt-4', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=8, total_tokens=17))


Adjusted function

In [27]:
import re
def correct_single_template(template, user_strings=None):
    """Apply all rules to process a template.

    DS (Double Space)
    BL (Boolean)
    US (User String)
    DG (Digit)
    PS (Path-like String)
    WV (Word concatenated with Variable)
    DV (Dot-separated Variables)
    CV (Consecutive Variables)

    """

    boolean = {'true', 'false'}
    default_strings = {'null', 'root', 'admin'}
    path_delimiters = {  # reduced set of delimiters for tokenizing for checking the path-like strings
        r'\s', r'\,', r'\!', r'\;', r'\:',
        r'\=', r'\|', r'\"', r'\'',
        r'\[', r'\]', r'\(', r'\)', r'\{', r'\}'
    }
    token_delimiters = path_delimiters.union({  # all delimiters for tokenizing the remaining rules
        r'\.', r'\-', r'\+', r'\@', r'\#', r'\$', r'\%', r'\&',
    })

    if user_strings:
        default_strings = default_strings.union(user_strings)

    # apply DS
    template = template.strip()
    template = re.sub(r'\s+', ' ', template)

    r'\{\{.*?\}\}'

    # apply PS (Path-like String)
    # p_tokens = re.split('(' + '|'.join(path_delimiters) + ')', template)
    # new_p_tokens = []
    # for p_token in p_tokens:
    #     if re.match(r'^(\/[^\/]+)+$', p_token):
    #         p_token = '<*>'
    #     new_p_tokens.append(p_token)
    # template = ''.join(new_p_tokens)

    # tokenize for the remaining rules
    # tokens = re.split('(' + '|'.join(token_delimiters) + ')',
    #                   template)  # tokenizing while keeping delimiters
    new_tokens = []
    for token in tokens:
        # apply BL, US
        # for to_replace in boolean.union(default_strings):
        #     if token.lower() == to_replace.lower():
        #         token = '<*>'

        # apply DG
        # if re.match(r'^\d+$', token):
        #     token = '<*>'

        # apply WV
        if re.match(r'^[^\s\/]*<\*>[^\s\/]*$', token):
            if token != '<*>/<*>':  # need to check this because `/` is not a deliminator
                token = '<*>'

        # collect the result
        new_tokens.append(token)

    # make the template using new_tokens
    template = ''.join(new_tokens)

    # Substitute consecutive variables only if separated with any delimiter including "." (DV)
    while True:
        prev = template
        template = re.sub(r'<\*>\.<\*>', '<*>', template)
        if prev == template:
            break

    # Substitute consecutive variables only if not separated with any delimiter including space (CV)
    # NOTE: this should be done at the end
    while True:
        prev = template
        template = re.sub(r'<\*><\*>', '<*>', template)
        if prev == template:
            break

    while "#<*>#" in template:
        template = template.replace("#<*>#", "<*>")

    while "<*>:<*>" in template:
        template = template.replace("<*>:<*>", "<*>")
    return template

def parse_batch_log_template(response, logs=None):
    if "\n\n" in response:
        response = response.split("\n\n")[0]
    reg = re.compile("`([^`]+)`")
    tmps = reg.findall(response)
    tmps = [x.strip('\n').strip() for x in tmps]
    if len(tmps) == 0:
        return ''
    elif len(tmps) == 1:
        return tmps[0]
    else:
        return max(tmps, key=len)


# `updateLightsLocked,mInCall ={boolean},mScreenOn = {boolean},ledNotification == null?{boolean}`

input = '''`Finished task {{task_id}} in stage {{stage_id}} (TID {{task_tid}}). {{bytes}} bytes result sent to driver`'''
print(correct_single_template(parse_batch_log_template(input)))
# parse_batch_log_template(input)

Finished task {{task_id}} in stage {{stage_id}} (TID {{task_tid}}). {{bytes}} bytes result sent to driver


In [30]:
re.sub(r'(?i)(?<![a-z])true(?![a-z])', '<*>', ':True', flags=re.IGNORECASE)

':<*>'